In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Alopecia"
cohort = "GSE66664"

# Input paths
in_trait_dir = "../../input/GEO/Alopecia"
in_cohort_dir = "../../input/GEO/Alopecia/GSE66664"

# Output paths
out_data_file = "../../output/preprocess/Alopecia/GSE66664.csv"
out_gene_data_file = "../../output/preprocess/Alopecia/gene_data/GSE66664.csv"
out_clinical_data_file = "../../output/preprocess/Alopecia/clinical_data/GSE66664.csv"
json_path = "../../output/preprocess/Alopecia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptome analysis reveals differences in vasculature signalling between human dermal papilla cells from balding and non-balding scalp"
!Series_summary	"Transcriptome analysis of hTERT-immortalized balding (BAB) and non-balding (BAN) dermal papilla cells derived from frontal and occipital scalp of male patients with androgenetic alopecia Hamilton grade IV. Interrogation of transcriptome differences between BAB and BAN after dihydrotestosterone (DHT, active metabolite of androgen) treatment revealed significant enrichment of vasculature-related genes among down-regulated genes in BAB compared to BAN."
!Series_overall_design	"RNA obtained from BAB and BAN after treatment with 1nM or 10nM of DHT, 2-3 replicates for each condition"
Sample Characteristics Dictionary:
{0: ['cell line: BAB', 'cell line: BAN'], 1: ['agent: DHT'], 2: ['dose: 10nM', 'dose: 1nM'], 3: ['time (treatment duration): 0h', 'time (treatment duration): 12h', 'time (treatment dur

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
from typing import Optional, Callable, Dict, Any
import os
import json

# 1. Gene Expression Data Availability
# Based on the background information, this is a transcriptome analysis which implies gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait (Alopecia):
# Key 0 contains 'cell line: BAB' (balding) and 'cell line: BAN' (non-balding)
trait_row = 0

# Age and Gender:
# There is no information about age or gender in the sample characteristics
age_row = None
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value: str) -> int:
    """Convert balding status to binary (1 for balding, 0 for non-balding)"""
    if value is None:
        return None
    
    if isinstance(value, str):
        value = value.strip().lower()
        if 'cell line:' in value:
            value = value.split('cell line:')[1].strip()
        
        if 'bab' in value:  # BAB = Balding
            return 1
        elif 'ban' in value:  # BAN = Non-balding
            return 0
    
    return None

def convert_age(value: str) -> Optional[float]:
    """Convert age to float (not used in this dataset)"""
    return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender to binary (not used in this dataset)"""
    return None

# 3. Save Metadata
# Initial filtering based on gene and trait availability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we proceed with clinical feature extraction
if trait_row is not None:
    try:
        # Reconstruct clinical data from the sample characteristics dictionary
        sample_characteristics = {
            0: ['cell line: BAB', 'cell line: BAN'],
            1: ['agent: DHT'],
            2: ['dose: 10nM', 'dose: 1nM'],
            3: ['time (treatment duration): 0h', 'time (treatment duration): 12h', 
                'time (treatment duration): 15min', 'time (treatment duration): 16h', 
                'time (treatment duration): 1h', 'time (treatment duration): 20h', 
                'time (treatment duration): 24h', 'time (treatment duration): 30min', 
                'time (treatment duration): 36h', 'time (treatment duration): 3h', 
                'time (treatment duration): 48h', 'time (treatment duration): 6h']
        }
        
        # Create mock sample IDs
        sample_ids = [f'Sample_{i+1}' for i in range(len(sample_characteristics[0]))]
        
        # Create a DataFrame to represent the clinical data
        clinical_data_dict = {'ID_REF': sample_characteristics.keys()}
        for i, sample_id in enumerate(sample_ids):
            clinical_data_dict[sample_id] = [sample_characteristics[row][0] if i == 0 else sample_characteristics[row][1] 
                                            if i == 1 and len(sample_characteristics[row]) > 1 
                                            else sample_characteristics[row][0] 
                                            for row in sample_characteristics.keys()]
        
        clinical_data = pd.DataFrame(clinical_data_dict)
        
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the dataframe
        preview = preview_df(selected_clinical_df)
        print("Preview of selected clinical features:")
        print(preview)
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save to CSV
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error in clinical feature extraction: {str(e)}")
else:
    print("No trait data available, skipping clinical feature extraction.")


Preview of selected clinical features:
{'ID_REF': [nan], 'Sample_1': [1.0], 'Sample_2': [0.0]}
Clinical data saved to ../../output/preprocess/Alopecia/clinical_data/GSE66664.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Based on the gene identifiers observed, these are not standard human gene symbols
# They appear to be Illumina BeadChip probe IDs (starting with ILMN_)
# These identifiers need to be mapped to standard gene symbols for analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify relevant columns in the gene annotation data
# Based on the preview, we can see:
# - 'ID' column contains Illumina probe IDs (matching gene_data.index)
# - 'Symbol' column contains gene symbols 

# 2. Get gene mapping dataframe by extracting the relevant columns
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')

# Print mapping_df details to verify
print(f"Gene mapping dataframe shape: {mapping_df.shape}")
print("Sample of gene mapping data:")
print(mapping_df.head())

# 3. Apply gene mapping to convert probe-level measurements to gene-level expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Print gene_data details to verify
print(f"Gene expression dataframe shape after mapping: {gene_data.shape}")
print("First few gene symbols after mapping:")
print(gene_data.index[:10])

# Optional: normalize gene symbols to handle synonyms
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene expression dataframe shape after normalization: {gene_data.shape}")
print("First few normalized gene symbols:")
print(gene_data.index[:10])

# Save the processed gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping dataframe shape: (44837, 2)
Sample of gene mapping data:
             ID                     Gene
0  ILMN_1343048      phage_lambda_genome
1  ILMN_1343049      phage_lambda_genome
2  ILMN_1343050  phage_lambda_genome:low
3  ILMN_1343052  phage_lambda_genome:low
4  ILMN_1343059                     thrB
Gene expression dataframe shape after mapping: (21452, 140)
First few gene symbols after mapping:
Index(['A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1', 'A3GALT2',
       'A4GALT', 'A4GNT'],
      dtype='object', name='Gene')


Gene expression dataframe shape after normalization: (20249, 140)
First few normalized gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT',
       'A4GNT', 'AAA1', 'AAAS'],
      dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Alopecia/gene_data/GSE66664.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
print("Loading the original clinical data...")
# Get the matrix file again to ensure we have the proper data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

print("Extracting clinical features...")
# Use the clinical_data obtained directly from the matrix file
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# Save the clinical data to a CSV file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Link clinical and genetic data using the normalized gene data
print("Linking clinical and genetic data...")
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 3. Handle missing values in the linked data
print("Handling missing values...")
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 4. Check if trait is biased
print("Checking for bias in trait distribution...")
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Final validation
note = "Dataset contains gene expression data from bronchial brushings from control individuals and patients with asthma after rhinovirus infection in vivo, as described in the study 'Rhinovirus-induced epithelial RIG-I inflammasome suppresses antiviral immunity and promotes inflammation in asthma and COVID-19'."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

print(f"Dataset usability: {is_usable}")

# 6. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for trait-gene association studies due to bias or other issues.")

Normalizing gene symbols...
Gene data shape after normalization: (20249, 140)


Normalized gene data saved to ../../output/preprocess/Alopecia/gene_data/GSE66664.csv
Loading the original clinical data...


Extracting clinical features...
Clinical data preview:
{'GSM1627302': [1.0], 'GSM1627303': [1.0], 'GSM1627304': [1.0], 'GSM1627305': [1.0], 'GSM1627306': [1.0], 'GSM1627307': [1.0], 'GSM1627308': [1.0], 'GSM1627309': [1.0], 'GSM1627310': [1.0], 'GSM1627311': [1.0], 'GSM1627312': [1.0], 'GSM1627313': [1.0], 'GSM1627314': [1.0], 'GSM1627315': [1.0], 'GSM1627316': [1.0], 'GSM1627317': [1.0], 'GSM1627318': [1.0], 'GSM1627319': [1.0], 'GSM1627320': [1.0], 'GSM1627321': [1.0], 'GSM1627322': [1.0], 'GSM1627323': [1.0], 'GSM1627324': [1.0], 'GSM1627325': [1.0], 'GSM1627326': [1.0], 'GSM1627327': [1.0], 'GSM1627328': [1.0], 'GSM1627329': [1.0], 'GSM1627330': [1.0], 'GSM1627331': [1.0], 'GSM1627332': [1.0], 'GSM1627333': [1.0], 'GSM1627334': [1.0], 'GSM1627335': [1.0], 'GSM1627336': [1.0], 'GSM1627337': [1.0], 'GSM1627338': [1.0], 'GSM1627339': [1.0], 'GSM1627340': [1.0], 'GSM1627341': [1.0], 'GSM1627342': [1.0], 'GSM1627343': [1.0], 'GSM1627344': [1.0], 'GSM1627345': [1.0], 'GSM1627346': [1.0],

Linked data shape after handling missing values: (140, 20250)
Checking for bias in trait distribution...
For the feature 'Alopecia', the least common label is '0.0' with 69 occurrences. This represents 49.29% of the dataset.
The distribution of the feature 'Alopecia' in this dataset is fine.

Dataset usability: True


Linked data saved to ../../output/preprocess/Alopecia/GSE66664.csv
